In [54]:
# Getting the credentials from the es_config.txt file
d = {i.split('=')[0]:i.split('=')[1] for i in  [line.rstrip() for line in open('./es_config.txt')]}


# Importing required packages
try:
    from elasticsearch import Elasticsearch
    import boto3

except Exception as e:
    print(f"Error occured while importing required packages - {e}")

# Creating ES and s3 client
try:
    es = Elasticsearch(d['ES_HOST'], basic_auth=('elastic',d['ES_PASSWORD']))
    session = boto3.Session(aws_access_key_id=d['AWS_ACCESS_KEY_ID'], aws_secret_access_key=d['AWS_SECRET_ACCESS_KEY'])
    s3 = session.client('s3')
except Exception as e:
    print(f"Error occured while creating ES/S3 client - {e}")



In [55]:
response = s3.list_objects_v2(Bucket=d['BUCKET_NAME'])
files = []
for obj in response['Contents']:
    files.append(obj['Key'])

In [56]:
files

['2020/aug/2020_07_02/2020_07_02_0.json',
 '2020/aug/2020_07_02/2020_07_02_1.json',
 '2020/aug/2020_07_02/2020_07_02_2.json',
 '2020/aug/2020_07_02/2020_07_02_3.json',
 '2020/aug/2020_07_02/2020_07_02_4.json',
 '2020/aug/2020_07_02/2020_07_02_5.json',
 '2020/aug/2020_07_02/2020_07_02_6.json',
 '2020/aug/2020_07_02/2020_07_02_7.json',
 '2020/aug/2020_07_02/2020_07_02_8.json',
 '2020/aug/2020_07_02/2020_07_02_9.json',
 '2020/aug/2020_07_03/2020_07_03_0.json',
 '2020/aug/2020_07_03/2020_07_03_1.json',
 '2020/aug/2020_07_03/2020_07_03_2.json',
 '2020/aug/2020_07_03/2020_07_03_3.json',
 '2020/aug/2020_07_03/2020_07_03_4.json',
 '2020/aug/2020_07_03/2020_07_03_5.json',
 '2020/aug/2020_07_03/2020_07_03_6.json',
 '2020/aug/2020_07_03/2020_07_03_7.json',
 '2020/aug/2020_07_03/2020_07_03_8.json',
 '2020/aug/2020_07_03/2020_07_03_9.json',
 '2020/aug/2020_07_04/2020_07_04_0.json',
 '2020/aug/2020_07_04/2020_07_04_1.json',
 '2020/aug/2020_07_04/2020_07_04_2.json',
 '2020/aug/2020_07_04/2020_07_04_3

In [52]:
#es.indices.delete(index='amx')

{'acknowledged': True}

In [57]:
es.indices.exists(index="amx")

False

In [58]:
es.indices.create(index='amx',ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'amx'}

In [59]:
#Creating Object From the S3 Resource.  

import json 
obj = s3.get_object(Bucket=d['BUCKET_NAME'], Key='2020/aug/2020_07_02/2020_07_02_0.json')
j = json.loads(obj['Body'].read())

In [40]:
j.keys()

dict_keys(['status', 'numResults', 'articles'])

In [60]:
len(j['articles'])

1000

In [61]:
for i in range(0,len(j['articles'])):
    #print(j['articles'][0])
    es.index(index='amx', body=j['articles'][i])


In [63]:
es.cat.count(index="amx", format="json")

[{'epoch': '1683157957', 'timestamp': '23:52:37', 'count': '1000'}]

Implement Bulk Load

In [64]:
from elasticsearch import helpers

In [65]:
obj = s3.get_object(Bucket=d['BUCKET_NAME'], Key='2020/aug/2020_07_02/2020_07_02_4.json')
j = json.loads(obj['Body'].read())

In [67]:
type(j['articles'][0])

dict

In [69]:
j['articles'][0].keys()

dict_keys(['url', 'authorsByline', 'articleId', 'clusterId', 'source', 'imageUrl', 'country', 'language', 'pubDate', 'addDate', 'refreshDate', 'score', 'title', 'description', 'content', 'medium', 'labels', 'matchedAuthors', 'claim', 'verdict', 'keywords', 'topics', 'categories', 'entities', 'sentiment', 'summary', 'translation', 'locations', 'reprint', 'places'])

In [70]:
j['articles'][1].keys()

dict_keys(['url', 'authorsByline', 'articleId', 'clusterId', 'source', 'imageUrl', 'country', 'language', 'pubDate', 'addDate', 'refreshDate', 'score', 'title', 'description', 'content', 'medium', 'labels', 'matchedAuthors', 'claim', 'verdict', 'keywords', 'topics', 'categories', 'entities', 'sentiment', 'summary', 'translation', 'locations', 'reprint', 'places'])

In [71]:
resp = helpers.bulk(es,
j['articles'],
index = "amx",
doc_type = "_doc"
)

d:\ALPHA\AMX-Backend-V2\.venv\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [72]:
es.cat.count(index="amx", format="json")

[{'epoch': '1683159350', 'timestamp': '00:15:50', 'count': '2000'}]

In [ ]:
query={
        "query": {
            "match_all": {}
        }
    }

In [ ]:
resp = es.search(index='amx',body=query)